In [9]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, start_yr, end_yr):
    for yr in range(start_yr, end_yr+1):        
        
        file = os.path.join(out_forc_name_base + '.' + str(yr) + '.nc')
        f=xr.open_dataset(file)
        time = f['time'][:]
        pcp_error = f.variables['pcp_error_update'][:]
        pcp_error_2 = f.variables['pcp_error_2_update'][:]
        tmean_error_2 = f.variables['tmean_error_2'][:]
        trange_error_2 = f.variables['trange_error_2'][:]
        
        if yr == start_yr:
            time_concat = time
            pcp_error_concat = pcp_error
            pcp_error_2_concat = pcp_error_2
            tmean_error_2_concat = tmean_error_2
            trange_error_2_concat = trange_error_2
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_error_concat = np.concatenate((pcp_error_concat, pcp_error), axis=0) # (time,y,x)
            pcp_error_2_concat = np.concatenate((pcp_error_2_concat, pcp_error_2), axis=0) 
            tmean_error_2_concat = np.concatenate((tmean_error_2_concat, tmean_error_2), axis=0)
            trange_error_2_concat = np.concatenate((trange_error_2_concat, trange_error_2), axis=0)
            
    time_concat = pd.DatetimeIndex(time_concat)
        
    return time_concat, pcp_error_concat, pcp_error_2_concat, tmean_error_2_concat, trange_error_2_concat

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
start_yr = 2013
end_yr = 2016

gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
scenarios_ids = range(0,6) #[0,1,5,8] 
intervals =  range(6,1,-1) #[10,9,5,2]
scenario_num = len(scenarios_ids)

subforlder = 'gmet_regr'
file_basename = 'regress_ts'

ens_num = 100
time_format = '%Y-%m-%d'

dpi_value = 300
plot_date_start = '2017-12-02'
plot_date_end = '2018-04-07'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step18_plot_regr_error_box')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename = 'step18_plot_regr_error_box.png'
    
# #======================================================================================================
# print('Read gridinfo mask')
# # get xy mask from gridinfo.nc
# f_gridinfo = xr.open_dataset(gridinfo_file)
# mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# #data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.

# #======================================================================================================
# # read scenario ensemble results and save to dictionary
# print('Read regression errors')

# for k in range(scenario_num):

#     test_folder = test_folders[scenarios_ids[k]]
    
#     print(test_folder)
#     test_dir = os.path.join(result_dir, test_folder)
#     fig_title= test_folder

#     print(' -- read spatial errors')
#     # read ensemble mean    
#     output_namebase = os.path.join(test_dir,subforlder, file_basename)
#     time_regr, pcp_error,  pcp_error_2, tmean_error_2, trange_error_2 = read_ens(output_namebase, start_yr, end_yr)
    
#     # define plot mask for nldas ensemble
#     mask_ens_t = (time_regr>=plot_date_start_obj) & (time_regr<=plot_date_end_obj)
    
#     print(' -- calculate temporal mean')
#     # caluclate time series mean(ny,nx)
#     pcp_error_mean = np.nanmean(pcp_error[mask_ens_t,:,:],axis=0)     
#     pcp_error_2_mean = np.nanmean(pcp_error_2[mask_ens_t,:,:],axis=0)     
#     tmean_error_2_mean = np.nanmean(tmean_error_2[mask_ens_t,:,:],axis=0)
#     trange_error_2_mean = np.nanmean(trange_error_2[mask_ens_t,:,:],axis=0)
    
#     print(' -- extract unmasked values')
#     # extract unmasked values
#     pcp_error_mean=pcp_error_mean[mask_xy!=0]    
#     pcp_error_2_mean=pcp_error_2_mean[mask_xy!=0]    
#     tmean_error_2_mean=tmean_error_2_mean[mask_xy!=0] 
#     trange_error_2_mean=trange_error_2_mean[mask_xy!=0] 
    
#     # save to array
#     if k == 0:
#         grid_num = len(pcp_error_mean)
#         pcp_error_mean_arr = np.zeros((grid_num,scenario_num))
#         pcp_error_2_mean_arr = np.zeros((grid_num,scenario_num))
#         tmean_error_2_mean_arr = np.zeros((grid_num,scenario_num)) 
#         trange_error_2_mean_arr = np.zeros((grid_num,scenario_num))
    
#     pcp_error_mean_arr[:,k] = pcp_error_mean
#     pcp_error_2_mean_arr[:,k] = pcp_error_2_mean
#     tmean_error_2_mean_arr[:,k] = tmean_error_2_mean 
#     trange_error_2_mean_arr[:,k] = trange_error_2_mean
    
#     del pcp_error_mean, pcp_error_2_mean, tmean_error_2_mean, trange_error_2_mean
#     del pcp_error, pcp_error_2, tmean_error_2, trange_error_2  

#======================================================================================================    
# plot
print('Plot')
var_list = ["Precp'", "Precp_2'", 'Tmean_2', 'Trange_2']

nrow = len(var_list) # prcp, tmean, tmin, tmax, trange
ncol = 1 
fig, ax = plt.subplots(nrow, ncol, figsize=(5.5,5.5*1.2))#, constrained_layout=True)

for i in range(nrow):
        print(var_list[i])
        
        # select data for each subplot
        if i == 0:
            data=pcp_error_mean_arr
            top=2
        elif i == 1:
            data=pcp_error_2_mean_arr
            top=0.7
        elif i == 2:
            data=tmean_error_2_mean_arr
            top=18
        elif i == 3:
            data=trange_error_2_mean_arr
            top=15
        
        # boxplot
        # reference: https://matplotlib.org/3.1.1/gallery/statistics/boxplot_demo.html
        bp = ax[i].boxplot(data, sym='o')#, labels=labels)
        plt.setp(bp['boxes'], color='black')
        plt.setp(bp['whiskers'], color='black')
        plt.setp(bp['fliers'], color='red', marker='o',markersize=1.5)
        
        # Add a horizontal grid to the plot, but make it very light in color
        # so we can use it for reading data values but not be distracting
        ax[i].yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
        ax[i].set_axisbelow(True)
        
        # y_lim
#         bottom=np.nanmin(data)-0.05*(np.nanmax(data)-np.nanmin(data))
#         top=np.nanmax(data)*1.05
        ax[i].set_ylim(bottom=0, top=top)

        # Due to the Y-axis scale being different across samples, it can be
        # hard to compare differences in medians across the samples. Add upper
        # X-axis tick labels with the sample medians to aid in comparison
        # (just use two decimal places of precision)
        pos = np.arange(scenario_num) + 1
        medians = [(bp['medians'][k]).get_ydata()[0] for k in range(scenario_num)]
        upper_labels = [str(np.round(s, 2)) for s in medians]
        for tick, label in zip(range(scenario_num), ax[i].get_xticklabels()):
            k = tick % 2
            ax[i].text(pos[tick]+0.2, 0.9, upper_labels[tick],
                     transform=ax[i].get_xaxis_transform(),
                     horizontalalignment='center', size='xx-small',
                     fontstyle='italic', color='b') #pos[tick], 1.02

        # set y-axis label
        y_lable = 'Regression uncertainty'
        ax[i].set_ylabel(y_lable, fontsize='xx-small')
        if i == nrow-1:
            ax[i].set_xlabel('Sampling Scenario', fontsize='xx-small')
        
        ax[i].tick_params(axis='both', direction='out',labelsize = 'xx-small',
                          length=1.5, width=0.5, pad=1.5)
        
        # title
        alpha = chr(ord('a') + i)
        ax[i].set_title('('+alpha+') '+var_list[i], pad=4, 
                        fontsize='xx-small', fontweight='semibold') #pad=9
        
#         # change subplot border width
#         for axis in ['top','bottom','left','right']:
#             ax[i,k].spines[axis].set_linewidth(0.5)

# save plot
fig.tight_layout(pad=0.1, h_pad=0.5) #h_pad=0.7
fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value, bbox_inches = 'tight', pad_inches = 0.05)
plt.close(fig)

print('Done')


Plot
Precp'
Precp_2'
Tmean_2
Trange_2
Done


In [3]:
a=(pcp_ensub[mask_ens_t,:,:]-pcp_enslb[mask_ens_t,:,:])
np.shape(a)

(731, 224, 464)